In [ ]:
import gzip
import numpy as np

In [ ]:
# Преобразуем всё в три последовательности строчек вида 
# idx - sequence - secstr

counter = 0
s = []
d = {'idx': [], 'sequence': [], 'secstr': []}
with gzip.open('./ss.txt.gz', 'rt') as f:
    entity = []
    cur_type = None
    for line in f:
        counter += 1
        if line.startswith('>'):
            if s:
                s = ''.join(s)
                d[cur_type] += [s]
            s = []
            idx1, idx2, cur_type = line[1:-1].split(':')
            if cur_type == 'sequence':
                d['idx'] += ['{}:{}'.format(idx1, idx2)]
        else:
            s += [line.rstrip('\n')]
#         if counter > 30000:
#             break
    s = ''.join(s)
    d[cur_type] += [s]

counter

4853004

In [ ]:
# Посмотрим, сколько уникальных символов в каждом виде последовательностей:
unique_dict = {'sequence': set(), 'secstr': set()}
for key in ['sequence', 'secstr']:
    for s in d[key]:
        unique_dict[key].update(set(s))
for key in ['sequence', 'secstr']:
    unique_dict[key] = list(unique_dict[key])
    
len(unique_dict['sequence']), len(unique_dict['secstr'])
# Обнаруживаем, что есть выбросы в типах аминокислот, поэтому посчитаем частоты встречаемости

(25, 8)

In [ ]:
unique_dict['secstr']

['E', 'H', 'B', 'S', 'T', 'G', ' ', 'I']

In [ ]:
amino = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
dict_amino = {}
for idx, a in enumerate(amino):
    dict_amino[a] = idx
counter_amino = {}

for s in d['sequence']:
    for c in s:
        counter_amino[c] = counter_amino.get(c, 0) + 1

In [ ]:
counter_amino

{'M': 2996227,
 'V': 8965339,
 'L': 11597317,
 'S': 8170016,
 'E': 8422064,
 'G': 9410902,
 'W': 1708301,
 'Q': 4879679,
 'H': 3399868,
 'A': 10187616,
 'K': 7523327,
 'D': 7185421,
 'I': 7179674,
 'R': 6591792,
 'F': 5016056,
 'P': 5936867,
 'T': 7166597,
 'Y': 4386021,
 'N': 5385984,
 'C': 1738735,
 'X': 403758,
 'U': 110,
 'Z': 58,
 'B': 34,
 'O': 6}

In [ ]:
sec_list = list(unique_dict['secstr'])

counter_sec = {}

for s in d['secstr']:
    for c in s:
        counter_sec[c] = counter_sec.get(c, 0) + 1
        
counter_sec

{' ': 34280761,
 'H': 39337412,
 'G': 4193184,
 'T': 13289651,
 'S': 10955277,
 'E': 24715331,
 'B': 1450906,
 'I': 29247}

In [ ]:
# Теперь удалим все выбросы: оставим только 20 букв

def check_outlier(s, keys):
    # True — outlier
    res = False
    for key in keys:
        res = res or key in s
    return res

counter = 0
s = []
d = {'idx': [], 'sequence': [], 'secstr': []}
is_outlier = False
with gzip.open('./ss.txt.gz', 'rt') as f:
    entity = []
    cur_type = None
    for line in f:
        counter += 1
        if line.startswith('>'):
            if s:
                s = ''.join(s)
                if cur_type == 'sequence':
                    if not check_outlier(s, ['X', 'U', 'Z', 'B', 'O']):
                        d[cur_type] += [s]
                        d['idx'] += ['{}:{}'.format(idx1, idx2)]
                        is_outlier = False
                    else:
                        is_outlier = True
                elif not is_outlier:
                    d[cur_type] += [s]
            s = []
            idx1, idx2, cur_type = line[1:-1].split(':')
        else:
            s += [line.rstrip('\n')]
    if not is_outlier:
        s = ''.join(s)
        d[cur_type] += [s]

counter

4853004

In [ ]:
len(d['idx'])

467711

In [ ]:
np.unique(list(map(lambda x: x.split(':')[-1], d['idx'])), return_counts=True)

(array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C',
        'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P',
        'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c',
        'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
        'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'], dtype='<U1'),
 array([   509,   1038,   1041,    877,    749,    518,    482,    430,
           346,    269, 155950,  90013,  42228,  35334,  18254,  15897,
         10953,  12843,   7303,   6297,   5647,   8306,   4426,   3939,
          3688,   4214,   3210,   3194,   3009,   2798,   2487,   2123,
          1953,   2920,   1908,   1740,    995,    984,    639,    610,
           614,    607,    523,    525,    447,    413,    414,    412,
           387,    337,    347,    287,    275,    259,    222,    274,
           252,    227,    173,    203,    200,    192]))

In [ ]:
# One-hot encoding последовательностей

window_size = 10

def compute_one_hot(s, padding=0):
    # One-hot с padding`ом с двух сторон
    res = np.zeros((len(s) + 2 * padding, len(amino)), dtype=np.bool)
    for idx, c in enumerate(s):
        res[idx + padding, dict_amino[c]] = 1
    return res

In [ ]:
n_samples = 0
for sec in d['secstr']:
    n_samples += len(sec)
n_samples

126717436

In [ ]:
len(d['secstr']), len(d['sequence'])

(467711, 467711)

In [ ]:
lim = 4000

n_samples = 0
for sec in d['secstr'][:lim]:
    n_samples += len(sec)
n_samples

902329

In [ ]:
# Теперь формируем матрицу X, число признаков равно 2 * window_size
from scipy.sparse import csr_matrix, lil_matrix

# Разреженные матрицы, иначе не влезает в память:
X = np.zeros((n_samples, 2 * window_size * len(amino)), dtype=np.bool)
y = np.zeros(n_samples, dtype=np.str)
counter = 0
for seq, sec in zip(d['sequence'][:lim], d['secstr'][:lim]):
    onehot = compute_one_hot(seq, padding=window_size)
    for idx, c in enumerate(sec):
        X[counter] = onehot[idx:idx+2*window_size].ravel()
        y[counter] = c
        counter += 1

In [ ]:
from sklearn.svm import SVC, LinearSVC
# from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

perm = np.random.permutation(len(y))
train_size = int(0.8 * len(perm))
train_idx = perm[:train_size]
test_idx = perm[train_size:]

In [ ]:
svm = SVC(kernel='rbf', C=1., gamma=0.125, max_iter=100)
svm.fit(X[train_idx], y[train_idx])

/Users/talgatdaulbaev/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


SVC(gamma=0.125, max_iter=100)

In [ ]:
y_pred = svm.predict(X[test_idx])

In [ ]:
print('Accuracy:', np.sum(y[test_idx] == y_pred) / len(y_pred))

Accuracy: 0.2632961333436769
